In [7]:
import pandas as pd
import numpy as np

In [8]:
train = pd.read_csv("../data/3_5/train_preprocessed_onehot.csv")
train_x = train.drop(["target"], axis=1)
train_y = train["target"]

test_x = pd.read_csv("../data/3_5/test_preprocessed_onehot.csv")

train_x_saved = train_x.copy()
test_x_saved = test_x.copy()

In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 標準化を行った学習データとテストデータを返す関数
def load_standarized_data():
    train_x, test_x = train_x_saved.copy(), test_x_saved.copy()

    scaler = StandardScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    test_x = scaler.transform(test_x)
    return pd.DataFrame(train_x), pd.DataFrame(test_x)


# MinMaxスケーリングを行った学習データとテストデータを返す関数
def load_minmax_scaled_data():
    train_x, test_x = train_x_saved.copy(), test_x_saved.copy()

    # Min-Max Scalingを行う
    scaler = MinMaxScaler()
    scaler.fit(pd.concat([train_x, test_x], axis=0))
    train_x = scaler.transform(train_x)
    test_x = scaler.transform(test_x)

    return pd.DataFrame(train_x), pd.DataFrame(test_x)

In [12]:
train

,age,sex,height,weight,product_0,product_1,product_2,product_3,product_4,product_5,...,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth,medical_info_c1_nan,medical_info_c2_nan,target
0,50,1,166.445608,65.016732,0,0,0,0,0,0,...,1,0,0,2015,2,3,24182,False,True,0
1,68,0,164.334615,56.544217,1,0,0,0,0,0,...,1,0,0,2015,5,9,24185,True,True,0
2,77,1,167.462917,54.242267,0,0,1,0,0,0,...,1,0,0,2016,2,13,24194,False,True,1
3,17,1,177.097725,71.147762,0,0,0,1,0,0,...,1,0,0,2015,7,6,24187,False,False,0
4,62,0,158.165788,65.240697,0,1,0,0,0,0,...,1,1,0,2016,9,17,24201,False,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,61,1,182.729800,73.393777,0,1,0,0,0,0,...,1,1,0,2015,10,21,24190,False,True,0
9996,33,0,167.701136,75.006529,0,0,0,0,0,0,...,1,1,0,2015,5,28,24185,False,True,0
9997,44,0,145.609998,47.739397,0,0,0,0,0,0,...,1,0,1,2016,2,29,24194,True,True,0
9998,34,0,165.796017,57.567695,0,0,0,0,0,0,...,1,1,0,2016,2,27,24194,False,True,0


## 次元削減手法

### PCA
- 多次元データに対する次元削減の最も代表的な手法
- 多次元データを分散の大きい方向から順に軸を取り直す手法
- 変数間の従属性（共分散が大きい）が大きい場合、より少数の成分で元のデータを表現できる
- 各特徴量が正規分布に従っている状態を仮定している
    - 正規分布から外れた分布から生成された特徴量では不適切
- SVD（特異値分解）とほぼ同じ意味

In [10]:
from sklearn.decomposition import PCA

In [11]:
train_x, test_x = load_standarized_data()

In [14]:
pca = PCA(n_components=5)
pca.fit(train_x)

train_x = pca.transform(train_x)
test_x = pca.transform(test_x)

In [15]:
train_x

array([[ 2.05891375, -0.36779758, -1.04006073,  1.39094049,  1.46802353],
       [ 0.79790466, -1.39150075, -1.07593836,  1.68535846, -0.12824928],
       [-0.11010861,  0.18506683, -1.05469726,  0.70940665, -0.07820673],
       ...,
       [-1.79675974, -1.96694265,  1.96590651,  1.77199665, -0.71949104],
       [-0.78512007, -0.65046887,  1.76977886, -0.96003533,  0.02093909],
       [ 2.14373273,  0.988635  ,  0.6567765 , -1.30606196, -0.17988786]])